## Materia: Robotica
## Grupo: 1
## Alumno: Rodriguez Torres Angel Adrian 
## Profesor: M.I. Erik Peña Gonzáles Villela
## Practica 0

# Práctica 0 Nodos y Tópicos

## Objetivo
Que los alumnos comprendan los conceptos básicos relacionados con los Nodos y los Tópicos, mediante la creación un de paquete con políticas en Python.

## Metas

- La creación de espacio de trabajo de ROS 2.
- Creación de un paquete con políticas de Python.
- Comprender los elementos que contiene el espacio de trabajo al montar un paquete en Python.
- La creación de un paquete que contenga un Nodo publicador, un Nodo Subcriptor y un Nodo Publicador Subcriptor.
- Programar un Nodo publicador y un Nodo Publicador subcriptor-publicador que reciba una velocidad en rpm y la convierta en $\frac{rad}{s}$.

## Previo
## Nodo (Node)
Imagina que estás construyendo un robot. Este robot tiene muchas tareas que hacer al mismo tiempo: leer sensores, mover motores, procesar imágenes, etc.

Un Nodo es un programa encargado de una sola de esas tareas. Son como los especialistas en una fábrica:

    Un nodo se encarga exclusivamente de leer los datos de la cámara.

    Otro nodo se encarga exclusivamente de controlar los motores de las ruedas.

    Un tercer nodo se encarga de calcular la ruta para evitar obstáculos.

Cada nodo es un programa independiente. Esta división de tareas hace que el sistema sea modular, fácil de depurar y reutilizable. Si el control de motores falla, solo necesitas revisar ese nodo en particular.

## Tópico (Topic)

Los nodos necesitan hablar entre sí. Por ejemplo, el nodo que calcula la ruta necesita saber qué ve la cámara.

Un Tópico es un canal de comunicación con nombre, como una estación de radio. Los nodos pueden enviar (publicar) o recibir (suscribir) mensajes a través de estos canales.

    Un nodo publica un mensaje en un tópico. (Ej: El nodo de la cámara publica la imagen en el tópico /video_stream).

    Otro nodo se suscribe a ese tópico para recibir el mensaje. (Ej: El nodo de cálculo de ruta se suscribe a /video_stream para "ver" la imagen).
    
Esta comunicación es de "uno a muchos". Un nodo puede publicar en un tópico y muchos otros nodos pueden suscribirse para recibir la misma información. El publicador no sabe (ni le importa) quién está escuchando, simplemente envía la información al canal.

## ¿Qué es un Árbol de Directorios?

Imagina que es el índice de un libro, pero para tus archivos. Muestra qué carpetas están dentro de otras y qué archivos contiene cada una. Se usan líneas y sangrías para mostrar la estructura anidada.

### ¿Cómo se documenta la creación de paquetes? (Árbol)

La estructura de un paquete se documenta con un **árbol de directorios**. Es un esquema de texto que muestra la organización jerárququica de las carpetas y archivos del proyecto. Sirve para entender su estructura de un solo vistazo.

**Ejemplo de un paquete básico de Python en ROS 2:**
```
nombre_del_paquete/
├── package.xml
├── setup.py
└── nombre_del_paquete/
    ├── __init__.py
    └── nombre_del_nodo.py
```

Este árbol muestra los archivos de configuración (`package.xml`, `setup.py`) en la raíz y el código fuente de los nodos en una subcarpeta interna. 🌳

## Los tres tipos de interfaces que maneja ROS 2.
### 1. Mensajes (.msg)

Son para comunicación unidireccional. Un nodo publica (envía) datos a un tópico y no espera respuesta. Es como una transmisión de radio: el locutor habla y cualquiera con un radio sintonizado puede escuchar.

- **Tipo**: Publicar / Suscribir (Uno a muchos).
- **Ideal para**: Flujos continuos de datos, como las lecturas de un sensor, la posición de las articulaciones de un robot o el video de una cámara.
    
### 2. Servicios (.srv)

Son para comunicación bidireccional de tipo pregunta-respuesta. Un nodo "cliente" solicita algo y se detiene a esperar hasta que un nodo "servidor" le entrega una respuesta. Es una transacción rápida y directa.

- **Tipo**: Cliente / Servidor (Uno a uno).
- **Ideal para**: Tareas rápidas como "activa la cámara", "¿cuál es la distancia al obstáculo?", o pedir un cálculo matemático. Es como pedir un café: haces el pedido (pregunta) y esperas a que te lo entreguen (respuesta).

### 3. Acciones (.action)

Son para tareas de larga duración que necesitan dar retroalimentación y pueden ser canceladas. Un cliente envía un "objetivo" a un servidor, y el servidor va informando sobre el progreso hasta que completa la tarea y entrega un resultado.

- **Tipo**: Cliente / Servidor (con retroalimentación).
- **Ideal para**: Procesos que toman tiempo, como "navega hasta el punto B", "mueve el brazo robótico a esta posición" o "gira 360 grados". Es como pedir una pizza a domicilio: haces el pedido (objetivo), la app te avisa cuando está en el horno y en camino (retroalimentación), y finalmente la recibes (resultado).

## Desarrollo de la Práctica
El desarrollo de esta práctica consistió en la creación, construcción y ejecución de un paquete de ROS 2 llamado `nodes_topics_py`. El objetivo fue establecer un sistema de comunicación básico entre diferentes nodos utilizando tópicos.

### 1. Descripción del Sistema de Nodos
El sistema se compone de dos nodos principales que trabajan en cadena para procesar datos:

**Nodo Publicador (`publish_node_py`):** Este nodo actúa como un generador de datos. Su función es publicar mensajes de tipo entero de forma secuencial (0, 1, 2, ...) en un tópico llamado `/publish_topic`.

**Nodo Suscriptor-Publicador (`subscriber_node_py`):** Este nodo es el procesador del sistema. Realiza dos tareas:
* Se suscribe al tópico `/publish_topic` para recibir los números enviados por el primer nodo.
* Al recibir un número, lo procesa (en este caso, lo multiplica por 1.5) y lo vuelve a publicar como un mensaje de tipo decimal en un nuevo tópico llamado `/rad_vel_topic`.

Este flujo crea una tubería de datos simple: un nodo origina la información, un segundo la recibe, la transforma y la pone a disposición en otro tópico.

### 2. Proceso de Construcción del Paquete
Antes de poder ejecutar los nodos, es indispensable "construir" el paquete. Este proceso utiliza la herramienta `colcon` para leer los archivos de configuración (`package.xml` y `setup.py`), procesar el código fuente y generar los archivos ejecutables que ROS 2 necesita para poder lanzar los programas.

El comando se ejecuta desde la raíz del espacio de trabajo (`~/ROS2Dev`):

In [ ]:
colcon build

## Conclusiones

En esta sección deberan escribir las conclusiones de la práctica con base en el objetivo planteado y las metas que se deban cumplir para su realización.

## Bibliografía 
[1] Open Robotics. (2022). Understanding ROS 2 nodes. ROS 2 Documentation: Humble. Disponible en: https://docs.ros.org/en/humble/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Nodes/Understanding-ROS2-Nodes.html

[2] Open Robotics. (2022). Understanding ROS 2 topics. ROS 2 Documentation: Humble. Disponible en: https://docs.ros.org/en/humble/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Topics/Understanding-ROS2-Topics.html

[3] Open Robotics. (2022). Understanding ROS 2 services. ROS 2 Documentation: Humble. Disponible en: https://docs.ros.org/en/humble/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Services/Understanding-ROS2-Services.html

[4] Open Robotics. (2022). Understanding ROS 2 actions. ROS 2 Documentation: Humble. Disponible en: https://docs.ros.org/en/humble/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Actions/Understanding-ROS2-Actions.html

[5] Open Robotics. (2022). Creating a package. ROS 2 Documentation: Humble. Disponible en: https://docs.ros.org/en/humble/Tutorials/Beginner-Client-Libraries/Creating-A-Package/Creating-A-Package.html